![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 3: Advanced Scala
***Written by Adam Izraelevitz (adamiz@berkeley.edu)***

**Prerequesites**
- Module 1: Basic Scala
- Module 2: Basic Chisel ***(optional)***

## Table of Contents
In this tutorial, you will learn some advanced features of Scala which enable you to write more powerful and expressive Chisel generators. We will focus on the following Scala features:

**[Functional Programming](#funcprog)**
1. [Immutable vs. Mutable Data Structures](#data)  
2. [A Simple Example](#example)  
2. [Map](#map)  
2. [Anonymous Functions](#anonfunc)  
2. [Filter](#filter)  
2. [Reduce](#reduce)  
2. [FoldLeft](#foldleft)

**[Type System](#types)**
2. [Static Types](#static)
2. [Common Idioms](#idiom)
2. [Generic Types](#generic)

**[Implicits](#implicits)**
1. [Implicit Arguments](#implicitargs)
1. [Implicit Conversions](#implicitconvs)

**[Match/Case Statements](#match)**
1. [Value Matching](#valuematch)
1. [Type Matching](#typematch)

**[Object-Oriented Programming](#objprog)**
2. [Abstract Classes](#abstract)
2. [Traits](#traits)
2. [Objects](#objects)  
2. [Companion Objects](#compobj)
2. [Case Classes](#caseclass)
2. [Unapply](#unapply)

# Functional Programming<a name="funcprog"></a>

Functional programming enables progammers to operate on immutable datastructures in a concise and powerful way. Before we delve into this style of programming, we need to understand the difference between immutability and mutability.

## Immutable vs Mutable Datastructures<a name="data"></a>

A datastructure is immutable if, after it is created, it cannot be changed. Conversely, mutable datastructures can be changed after creation.

Like array's in C or ArrayBuffer's in Java, Scala has a corresponding mutable List-like datastructure, `ArrayBuffer[String]`. We can use various update and appending operations to change its contents.

In [2]:
import scala.collection.mutable //Import mutable collections library
val arraybuf = mutable.ArrayBuffer[String]("a", "b", "c")
println(arraybuf)
// Because ArrayBuffer is mutable, you can update or append values to arraybuf
arraybuf(0) = "A"
println(arraybuf)
arraybuf += "d"
println(arraybuf)

ArrayBuffer(a, b, c)
ArrayBuffer(A, b, c)
ArrayBuffer(A, b, c, d)


import scala.collection.mutable //Import mutable collections library

arraybuf: collection.mutable.ArrayBuffer[String] = ArrayBuffer("A", "b", "c", "d")
res1_5: collection.mutable.ArrayBuffer[String] = ArrayBuffer("A", "b", "c", "d")

Alternatively, we can use an immutable datastructure, `List[String]`. We cannot use update or appending operations, but instead must create new immutable datastructures containing our changes:

In [1]:
// Because List is immutable, l will always have the 3 elements "a", "b", and "c"
val list = List("a", "b", "c")
println(list)
// We cannot append a new value or update a value
// list(0) = "A" // Illegal
// list += "d" // Illegal

// Instead, we must create a new List with our appended or updated value
val lAppended = list ++ List("d") // ++ returns a new immutable List with both input lists appended
println(lAppended)
val lUpdated = List("A") ++ list.tail // tail returns a new List without the first element
println(lUpdated)

List(a, b, c)
List(a, b, c, d)
List(A, b, c)


list: List[String] = List("a", "b", "c")
lAppended: List[String] = List("a", "b", "c", "d")
lUpdated: List[String] = List("A", "b", "c")

Immutable datastructures have the benefits of eliminating side-effects, making your code's behavior more reliable and predictable.Scala style promotes using immutable datastructures whenever possible; the only exceptions are when a mutable datastructure enables more performant or significantly cleaner code.

## A Simple Example<a name="example"></a>

Generally, we use functional programming to cleanly and concisely create new immutable datastructures from other immutable datastructures.

Suppose we are given a sequence of strings, and want to return a new sequence with each string "doubled":
```scala
val sequence = List("a", "b", "c")
println(sequence.mkString(" ")) //"a b c"
...
val newSequence = ...
...
println(newSequence.mkString(" ")) //"aa bb cc"
```

Using a for-loop and a mutable list, we can mimic how one could do this in C or Java:

In [3]:
val sequence = List("a", "b", "c")
println(sequence.mkString(" "))

val doubleABuf = mutable.ArrayBuffer[String]()
for(letter <- sequence) {
    doubleABuf += letter + letter
}
println(doubleABuf.mkString(" "))

a b c
aa bb cc


sequence: List[String] = List("a", "b", "c")
doubleABuf: mutable.ArrayBuffer[String] = ArrayBuffer("aa", "bb", "cc")

However, using functional programming, we can do the same but much more concisely:

In [4]:
val sequence = List("a", "b", "c")
println(sequence.mkString(" "))

val doubleSeq = sequence.map(s => s + s)
println(doubleSeq.mkString(" "))

a b c
aa bb cc


sequence: List[String] = List("a", "b", "c")
doubleSeq: List[String] = List("aa", "bb", "cc")

What's going on here?? Let's break it down.

## Map<a name="map"></a>
First, let's rewrite the same example to be more less syntactically concise, but more readable to those unfamiliar with functional programming.

In [ ]:
val sequence = List("a", "b", "c")
println(sequence.mkString(" "))

def double(s: String): String = s + s
val doubleSeq = sequence.map(double)
println(doubleSeq.mkString(" "))

As you can see, the function `double` takes a `String` and returns a new "doubled" `String`. We then pass this function, as an argument, to another function called `map`. When called, map returns a new `List` containing the results of applying `double` to every element in `sequence`.

Let's try a different function, `triple`, which appends three copies of a string together:

In [5]:
val sequence = List("a", "b", "c")
println(sequence.mkString(" "))

def triple(s: String): String = ???
val tripleSeq = sequence.map(triple)
println(tripleSeq.mkString(" "))

a b c


: 

As you can see, we can easily change the function we pass the `map`, while abstracting away the actual traversing of the sequence.

## Anonymous Functions<a name="anonfunc"></a>
In our earliest example, we had the following syntax.
```scala
s => s + s
```
This is an anonymous function, or a function that has no name.Think of this like an object with a single method. When we call an anonymous function (e.g. passing it arguments), it calls its method on those arguments. In this case, our argument `s` is doubled.

The following example shows us creating an anonymous function, then calling it:

In [ ]:
val anon = {x: Int => x + x} // Scala usually requires anonymous functions to be wrapped in {}
val result = anon(10)
println(result)

You can see that this is very similar to a Scala method declaration, `def`. Generally, we use anonymous functions when they are simple and referenced only once, and use method declarations otherwise.

Now, you should be able to fully understand our functional programming example:

In [ ]:
println(sequence.mkString(" "))

val doubleSeq = sequence.map(s => s + s)
println(doubleSeq.mkString(" "))

Try writing your own anonymous function, so that the example returns a sequence of tripled strings:

In [ ]:
val tripleSeq = sequence.map(s => ???)
println(tripleSeq.mkString(" "))

There are lots of different syntaxes for declaring anonymous functions in Scala. The five examples below list many anonymous functions with identical semantics, but different syntaxes:

In [8]:
// First example: anon-function accepting one argument, referencing that argument once
// TODO talk about _
// TODO mark which ones you prefer
val seq1 = Seq(1, 2, 3)
seq1.map(_ + 1)
seq1.map { _ + 1 }
seq1.map(x => x + 1)
seq1.map{ x => x + 1 }
seq1.map { case x => x + 1 }


// Second example: anon-function accepting one argument, referenced that argument >1 time
// Note, you cannot use the `_` syntax in this scenario!
val seq2 = Seq(1, 2, 3)
// seq2.map(_ + _) // Illegal!
seq2.map(x => x + x)
seq2.map(x => x + x)
seq2.map { case x => x + x }

// Third example: anon-function accepting two arguments, each referenced one time
val seq3: Seq[Int] = Seq(1, 2, 3)
seq3.reduce(_ + _)
seq3.reduce { _ + _ }
seq3.reduce { (x: Int, y: Int) => x + y }

// Fourth example: anon-function accepting one argument and passing it to a function
val seq4 = Seq(1, 2, 3)
def inc(i: Int): Int = i + 1
seq4.map(inc(_))
seq4.map { inc(_) }
seq4.map(x => inc(x))
seq4.map { x => inc(x) }

// Fifth example: anon-function accepting one argument and passing it to series of nested functions
// Note, you cannot use the `_` syntax in this scenario!
val seq5 = Seq(1, 2, 3)
// seq5.map(inc(inc(_))) //Illegal!
seq5.map(x => inc(inc(x)))
seq5.map { x => inc(inc(x)) }

seq1: Seq[Int] = List(1, 2, 3)
res7_1: Seq[Int] = List(2, 3, 4)
res7_2: Seq[Int] = List(2, 3, 4)
res7_3: Seq[Int] = List(2, 3, 4)
res7_4: Seq[Int] = List(2, 3, 4)
res7_5: Seq[Int] = List(2, 3, 4)
seq2: Seq[Int] = List(1, 2, 3)
res7_7: Seq[Int] = List(2, 4, 6)
res7_8: Seq[Int] = List(2, 4, 6)
res7_9: Seq[Int] = List(2, 4, 6)
seq3: Seq[Int] = List(1, 2, 3)
res7_11: Int = 6
res7_12: Int = 6
res7_13: Int = 6
seq4: Seq[Int] = List(1, 2, 3)
defined function inc
res7_16: Seq[Int] = List(2, 3, 4)
res7_17: Seq[Int] = List(2, 3, 4)
res7_18: Seq[Int] = List(2, 3, 4)
res7_19: Seq[Int] = List(2, 3, 4)
seq5: Seq[Int] = List(1, 2, 3)
res7_21: Seq[Int] = List(3, 4, 5)
res7_22: Seq[Int] = List(3, 4, 5)

## Filter<a name="filter"></a>

Like `map`, the function `filter` takes a function as an argument, but this function returns true or false. Thus, calling `filter` will returns a new sequence containing only elements that return true.

The following code prints a new sequence that only contains numbers greater than 5:

In [ ]:
val numbers = Seq(1, 2, 10, 6, 3, 9)
val filtered = numbers.filter(i => i > 5)
println(filtered.mkString(" "))

## Reduce<a name="reduce"></a>

The function `reduce` will iteratively call its input function on two elements, returning a new element, until only one remains.

The following code prints the total sum of all integers in the sequence:

In [ ]:
val sum = numbers.reduce((a, b) => a + b)
println(sum)

## FoldLeft<a name="foldleft"></a>

Reductions have the problem of failing when the input sequence is empty:

In [ ]:
val empty = Seq[Int]()
empty.reduce((a, b) => a + b)

In these instances, we can use the function `foldLeft`. This function takes two arguments, an initial value and another function. This function takes two arguments - one is the value you are collecting, and the second is an element of the sequence, and it must return the new value you are collecting.

TODO talk about foldLeft having two different argument types. TODO describe other commonly useful manipulators. TODO show type signatures for these.

As it turns out, we can use `foldLeft` to implement our `map`, `reduce`, and `filter` examples!

In [ ]:
val mapped = sequence.foldLeft(Seq[String]()){
    (seq, element) => seq ++ Seq(element + element)
}
println(mapped.mkString(" "))

val reduced = numbers.foldLeft(0){
    (sum, element) => sum + element
}
println(reduced)

val filtered = numbers.foldLeft(Seq[Int]()){
    (seq, element) => if (element > 5) seq ++ Seq(element) else seq
}
println(filtered.mkString(" "))

As you can see, `foldLeft` is a very powerful construct, and well worth starting at these examples to fully understand them.

Other useful sequence manipulators can be found here: http://www.scala-lang.org/api/current/scala/collection/immutable/List.html

# Type System<a name="types"></a>
Scala is a strongly-typed programming language. This is a two-edged sword; on one hand, many programs that would compile and execute in Python (a dynamically-typed language) would fail at compile time in Scala. On the other hand, programs that compile in Scala will contain many fewer runtime errors than a similar Python program.

In this section, our goal is to familiarize you with types as a first class citizen in Scala. While initially you may feel you have limited productivity, you will soon learn to understand compile-time error messages and how to architect your programs with the type system in mind to catch more errors for you. 

## Static Types<a name="static"></a>
All objects in Scala have a type, which is usually the object's class. Let's see some:

In [9]:
println(10.getClass)
println(10.0.getClass)
println("ten".getClass)

int
double
class java.lang.String


When you declare your own class, it has an associated type.

In [10]:
class MyClass {
    def myMethod = ???
}
println(new MyClass().getClass)

class $sess.cmd9Wrapper$Helper$MyClass


defined class MyClass

While not required, it is HIGHLY recommended that you **define input and output types for all function declarations**. This will let the Scala compiler catch improper use of a function.

In [ ]:
def double(s: String): String = s + s
// Uncomment the code below to test it
// double("hi")      // Proper use of double
// double(10)        // Bad input argument!
// double("hi") / 10 // Inproper use of double's output!

Functions that don't return anything return type `Unit`.

In [12]:
var counter = 0
def increment(): Unit = {
    counter += 1
}
increment()

counter: Int = 1
defined function increment

## Common Idioms<a name="idiom"></a>
There are several canonical Scala types that are ideally suited for various use cases that you may encounter.

### Option
There times when a function sometimes returns a value, and sometimes does not. Instead of erroring when it cannot return a value, Scala has a mechanism to encode this in the type system.

In the following example, we have a map containing several key/value pairs. If we try to access a missing key/value pair, then we get a runtime error:

In [ ]:
val map = Map("a" -> 1)
val a = map("a")
println(a)
val b = map("b")
println(b)

However, `Map` provides a second way to access a key's value, through the **get** method. This method returns a value of abstract class `Option`. `Option` has two subclasses, `Some` and `None`:

In [17]:
val map = Map("a" -> 1)
val a = map.get("a")
println(a)
// TODO gets
println(a.get)
val b = map.get("b")
println(b)
// println(b.get)
println(b.getOrElse(100))

Some(1)
1
None
100


map: Map[String, Int] = Map("a" -> 1)
a: Option[Int] = Some(1)
b: Option[Int] = None

As you'll see in later sections, we can use Scala's match statement to base control flow on Scala types.

### Seq/List
Scala's default immutable array is List. However, its super class is Seq. Generally, it is recommended to use Seq everywhere, as its default underlying implementation is List. However, it makes functions generally more accepting of inputs that are list-like. The following example demonstrates how using Seq makes functions more generic:

In [ ]:
def sumList(l: List[Int]): Int = l.reduce(_ + _)
def sumSeq(s: Seq[Int]): Int = s.reduce(_ + _)

val array = Array(1, 2, 3)
// Uncomment code below to test it
// sumSeq(array)  // Legal!
// sumList(array) // Illegal!

## Generic Types<a name="generic"></a>
Scala's generic types (also known as polymorphism) is very complicated, especially when coupling it with inheritance.

This section will just get your toes wet; to understand more, checkout [this tutorial](https://twitter.github.io/scala_school/type-basics.html).

Classes can be polymorphic in their types. One good example are sequences, which require knowing what the type of the elements it contains.

In [18]:
val seq1 = Seq("1", "2", "3") // Type is Seq[String]

// TODO?
val seq1Explicit = Seq[String]("1", "2", "3")

val seq2 = Seq(1, 2, 3)       // Type is Seq[Int]
val seq3 = Seq(1, "2", true)  // Type is Seq[Any]

seq1: Seq[String] = List("1", "2", "3")
seq1Explicit: Seq[String] = List("1", "2", "3")
seq2: Seq[Int] = List(1, 2, 3)
seq3: Seq[Any] = List(1, 2, true)

Functions can also be polymorphic in their input or output types. The following example defines a function that times how long it takes to run a block of code. It is parameterized based on the return type of the block of code:

In [20]:
// TODO talk about : => T
def time[T](block: => T): T = {
    val t0 = System.nanoTime()
    val result = block
    val t1 = System.nanoTime()
    val timeMillis = (t1 - t0) / 1000000.0
    println(s"Block took $timeMillis milliseconds!")
    result
}

def addMillion(): Int = (1 to 1000000).reduce(_ + _)
val int = time(addMillion)
println(s"Add 1 through a million is $int")

def gotBeef(): String = (1 to 1000000).map(_.toHexString).filter(_.contains("beef")).last
val string = time(gotBeef)
println(s"The largest number under a million that has beef: $string")

Block took 24.253522 milliseconds!
Add 1 through a million is 1784293664
Block took 92.784253 milliseconds!
The largest number under a million that has beef: ebeef


defined function time
defined function addMillion
int: Int = 1784293664
defined function gotBeef
string: String = "ebeef"

# Implicits<a name="implicits"></a>

There are often times when you are programming that requires a lot of boilerplate code. To handle this use case, Scala introduced the notion of **implicits**, which allow the compiler to do some syntactic sugar for you. Because lots of things happen behind the scenes, implicits can appear very magical. This section breaks down some basic examples to explain what they are and where they are commonly used.

## Implicit Arguments<a name="implicitargs"></a>
# TODO CURRYING
At times, your code will require accessing a top-level variable of some sort from deep within a series of function calls. Instead of manually threading this variable through every function call, you can use implicit arguments to do it for you.

In the following example, we can pass the number of cats implicitly or explicitly.

In [24]:
implicit val numberOfCats = 3
// implicit val numberOfDogs = 5 // SAD!

def tooManyCats(nDogs: Int)(implicit nCats: Int): Boolean = nCats > nDogs
tooManyCats(2)    // Argument passed implicitly!
tooManyCats(2)(1) // Argument passed explicitly!

numberOfCats: Int = 3
defined function tooManyCats
res23_2: Boolean = true
res23_3: Boolean = false

What's happening here? First, we define an implicit value **numberOfCats**. In a given scope, **there can only be one implicit value of a given type**. Then, we define a function that takes two argument lists; the first is any explicit parameters, and the second are any implicit parameters. When we call **tooManyCats**, we either omit the second implicit argument list (letting the compiler find it for us), or explicitly provide an argument (which can be different than the implicit value).

The following are ways implicit arguments can *fail*:
- Two or more implicit values of a given type are defined in a scope
- If the compiler cannot find an implicit value necessary for a function call

## Implicit Conversions<a name="implicitconvs"></a>
Like implicit arguments, implicit functions (also known as **implicit conversions**) are used to reduce boilerplate code. More specifically, they are used to automatically convert one Scala object into another.

In the following example, we have two classes, `Animal` and `Human`. `Animal` has a `species` field, but `Human` does not. However, by implementing an implicit conversion, we can call `species` on a `Human`.

## TODO talk about where implicit conversions should and should not be used

In [26]:
class Animal(val name: String, val species: String)
class Human(val name: String)
implicit def human2animal(h: Human): Animal = new Animal(h.name, "Homo sapiens")
val me = new Human("Adam")
println(me.species)

Homo sapiens


defined class Animal
defined class Human
defined function human2animal
me: wrapper.wrapper.Human = $sess.cmd25Wrapper$Helper$Human@49883241

# Match/Case Statements<a name="match"></a>

The Scala *matching* concept is used throughout Chisel and needs to be part of any Chisel programmers basic understanding. Scala provides the match operator which supports:
- Simple testing for alternatives, something like a *C* switch statement
- More complex testing of ad-hoc combinations of values
- Taking actions based on the type of a variable when it's type is unknown or underspecified, for example when
  - variable is taken from a heterogeneous list ```val mixedList = List(1, "string", false)```
  - or variable is known to be a member of a super-class but not which specific sub-class it is.
- Extraction of sub strings of a string that are specified with a *regular expression*


## Value Matching<a name="valuematch"></a>

The following example, depending on the **value** of the variable we **match** on, we execute a different **case** statement:

In [ ]:
// y is an integer variable defined somewhere else in the code
val y = 7
/// ...
val x = y match {
  case 0 => "zero" // One common syntax, preferred if fits in one line
  case 1 =>        // Another common syntax, preferred if does not fit in one line.
      "one"        // Note the code block continues until the next case
  case 2 => {      // Another syntax, but curly braces are not required
      "two"
  }
  case _ => "many" // _ is a wildcard that matches all values
}
println("y is " + x)

The match operator checks possible values and for each case returns a string.  A couple of things to note:
- Each code block that follows a the ```=>``` operator continues until it reaches either the ending brace of the match or the next case statement.
- A match is searched in the order of the case statements, once a case statement has been matched, no other
checks against other case statements are made.
- The use of underscore as a wildcard, to handle any value not found.

Also, multiple variables can be matched at the same time. Here's a simple example of a truth table implemented with a match statement and tuple of values:

In [29]:
def animalType(biggerThanBreadBox: Boolean, meanAsCanBe: Boolean): String = {
  (biggerThanBreadBox, meanAsCanBe) match {
    case (true, true) => "wolverine"
    case (true, false) => "elephant"
    case (false, true) => "shrew"
    case (false, false) => "puppy"
  }
}
println(animalType(true, true))

wolverine


defined function animalType

<span style="color:blue">**Exercise:**</span> Use a **match statement** and **foldLeft** to return true if the sequence of strings contains "needle":

In [ ]:
def foundNeedle(seq: Seq[String]): Boolean = {
    // Your code goes here!
    ???
}
assert(foundNeedle(Seq("hay", "hay", "hay", "needle", "hay")) == true)
assert(foundNeedle(Seq("hay", "hay", "hay", "hay",    "hay")) == false)
println("SUCCESS")

## Type Matching<a name="typematch"></a>

## TODO | or syntax
## TODO collect polymorphic list examples

Scala is a strongly typed language, so the types of all objects are known during runtime. We can use **match statements** to use this type information to dictate control flow:

In [ ]:
val sequence = Seq("a", 1, 0.0)
sequence.foreach { x =>
    x match {
        case s: String => println(s"$x is a String")
        case s: Int    => println(s"$x is an Int")
        case s: Double => println(s"$x is a Double")
        case _ => println(s"$x is an unknown type!")
    }
}

Type matching has some limitations. Because Scala runs on the JVM, and the JVM does not maintain polymorphic types, you cannot match on them at runtime (because they are all erased). Note that the following example always matches the first case statement, because the `[String]`, `[Int]`, and `[Double]` polymorphic types are erased, and the case statements are **actually** matching on just a `Seq`.

In [ ]:
val sequence = Seq(Seq("a"), Seq(1), Seq(0.0))
sequence.foreach { x =>
    x match {
        case s: Seq[String] => println(s"$x is a String")
        case s: Seq[Int]    => println(s"$x is an Int")
        case s: Seq[Double] => println(s"$x is a Double")
    }
}

Note that generally, Scala compilers will give a warning if you implement code like the example above.


<span style="color:blue">**Exercise:**</span> Use a **match statement** and **map** to return a new sequence of strings, where non-string types are serialized as follows:

In [ ]:
def convertToStrings(seq: Seq[Any]): Seq[String] = {
    // Your code goes here!
    ???
}
val mixedList = List(1, "string", false, 1.57)
assert(convertToStrings(mixedList) == Seq("1", "string", "F", "unsupported type"))
println("SUCCESS")

# Object Oriented Programming<a name="objclass"></a>
This section outlines how Scala implements the object-oriented programming paradigm. So far you have already seen classes, but Scala also has the following features:
- Abstract classes
- Traits
- Objects
- Companion Objects
- Case Classes

## Abstract Classes<a name="abstract"></a>
Abstract classes are just like other programming language implementations. They can define many unimplemented values that subclasses must implement. Any object can only directly inherit from one parent abstract class.

In [ ]:
abstract class MyAbstractClass {
    def myFunction(i: Int): Int
    val myValue: String
}
class ConcreteClass extends MyAbstractClass {
    def myFunction(i: Int): Int = i + 1
    val myValue = "Hello World!"
}
// Uncomment below to test!
// val abstractClass = new MyAbstractClass() // Illegal! Cannot instantiate an abstract class
val concreteClass = new ConcreteClass()      // Legal!


## Traits<a name="traits"></a>
Traits are very similar to abstract classes in that they can define unimplemented values. However, they differ in two ways:
- a class can inherit from multiple traits
- a trait cannot have constructor parameters

Traits are how Scala implements multiple inheritance, as shown in the example below. `MyClass` extends from both traits `HasFunction` and `HasValue`:

In [38]:
trait HasFunction {
    def myFunction(i: Int): Int
}
trait HasValue {
    val myValue: String
    val myOtherValue = 100
}
class MyClass extends HasFunction with HasValue {
    override def myFunction(i: Int): Int = i + 1
    val myValue = "Hello World!"
}
// Uncomment below to test!
// val myTraitFunction = new HasFunction() // Illegal! Cannot instantiate a trait
// val myTraitValue = new HasValue()       // Illegal! Cannot instantiate a trait
val myClass = new MyClass()                // Legal!

defined trait HasFunction
defined trait HasValue
defined class MyClass
myClass: wrapper.wrapper.MyClass = $sess.cmd37Wrapper$Helper$MyClass@5acb5d55

In general, always use traits over abstract classes, unless you are certain you want to enforce the single-inheritance restriction of abstract classes.

## Objects<a name="objects"></a>
Like Java, Scala has classes you can define and instantiate.

## TODO reword this

Scala has a language feature for these singleton classes, called objects.

In [ ]:
object MyObject {
    def hi: String = "Hello World!"
    def apply(msg: String) = msg
}
println(MyObject.hi)
println(MyObject("This message is important!")) // equivalent to MyObject.apply(msg)

## Companion Objects<a name="compobj"></a>

When a class and an object share the same name and defined in the same file, the object is called a **companion object**.

Companion objects are usually used for the following reasons:
  1. to contain constants related to the class
  2. to execute code before/after the class constructor
  3. to create multiple constructors for a class

In the example below, we will instantiate a number of instances of Animal. We want each animal to have a name, and to know its order within all instantiations. Finally, if no name is given, it should get a default name.

In [ ]:
object Animal {
    val defaultName = "Bigfoot"
    private var numberOfAnimals = 0
    def apply(name: String): Animal = {
        numberOfAnimals += 1
        new Animal(name, numberOfAnimals)
    }
    def apply(): Animal = apply(defaultName)
}
class Animal(name: String, order: Int) {
  def info: String = s"Hi my name is $name, and I'm $order in line!"
}

val bunny = Animal.apply("Hopper") // Calls the Animal factory method
println(bunny.info)
val cat = Animal("Whiskers")       // Calls the Animal factory method
println(cat.info)
val yeti = Animal()                // Calls the Animal factory method
println(yeti.info)


*What's happening here?*
1. Our **Animal companion object** defines a constant relevant to ```class Animal```:
```scala
val defaultName = "Bigfoot"
```
1. It also defines a private mutable integer to keep track of the order of Animal instances:
```scala 
private var numberOfAnimals = 0
```
1. It defines two **apply** methods, which are known as **factory methods** in that they return instances of the **class Animal**. 
    1. The first creates an instance of Animal using only one argument, ```name```, and uses ```numberOfAnimals``` as well to call the Animal class constructor.
```scala
def apply(name: String): Animal = {
            numberOfAnimals += 1
            new Animal(name, numberOfAnimals)
}
```
    1. The second factory method requires no argument, and instead uses the default name to call the other apply method.
```scala
def apply(): Animal = apply(defaultName)
```
1. These factory methods can be called naively like this
```scala
val bunny = Animal.apply("Hopper")
```
which eliminates the need to use the new keyword, but the real magic is that the compiler assumes the apply method any time it sees parentheses applied to an instance or object:
```scala
val cat = Animal("Whiskers")
```
1. Factory methods, usually provided via companion objects, allow alternative ways to express instance creations, provide additional tests for constructor parameters, conversions, and eliminate the need to use the keyword ```new```. Note that you must call the companion object's `apply` method for `numberOfAnimals` to be incremented.

**Chisel uses many companion objects, like Module.** When you write the following:
```scala
val myModule = Module(new MyModule)
```
you are calling the **Module companion object**, so Chisel can run background code before and after instantiating 
```MyModule```.

## Case Classes<a name="caseclass"></a>
Case classes are a special type of Scala class that provides some cool additional features. They are very common in Scala programming, so this section outlines some of their useful features:
- Allows **external access** to the **class parameters**
- **Eliminates** the need to use **`new`** when instantiating the class
- Automatically creates an **unapply method** that supplies access to all of the class Parameters.
- Cannot be subclassed from

In the following example, we declare three different classes, `Nail`, `Screw`, and `Staple`.

In [ ]:
class Nail(length: Int) // Regular class
val nail = new Nail(10) // Requires the `new` keyword
// println(nail.length) // Illegal! Class constructor parameters are not by default externally visible

class Screw(val threadSpace: Int) // By using the `val` keyword, threadSpace is now externally visible
val screw = new Screw(2)          // Requires the `new` keyword
println(screw.threadSpace)

case class Staple(isClosed: Boolean) // Case class constructor parameters are, by default, externally visible
val staple = Staple(false)           // No `new` keyword required
println(staple.isClosed)

`Nail` is a regular class, and its parameters are not externally visible because we did not use the `val` keyword in the argument list. It also requires the `new` keyword when declaring an instance of `Nail`.

`Screw` is declared similarly to `Nail`, but includes `val` in the argument list. This allows its parameter, `threadSpace`, to be visible externally.

By using a case class, `Staple` gets the benefit of all its parameters being externally visible (without needing the `val` keyword).

In addition, `Staple` does not require using `new` when declaring a case class. This is because the Scala compiler automatically creates a companion object for every case class in your code, which contains an apply method for the case class.

## Unapply<a name="unapply"></a>
As it turns out, the companion object that is created for every case class also contains an **unapply** method, in addition to an **apply** method. What is an **unapply** method?

Scala unapply methods are another form of syntactic sugar that give match statements the ability to both match on types and **extract values** from those types during the matching.

Let's look at the following example. We have three types of Disney characters: `Hero`, `Villain`, and `SideKick`. We also defined two variables, `jungleBook` and `lionKing` which contain a list of Disney characters. Finally, we have two functions: `teaser`, which prints out a summary of the movie, and `sequel`, which creates a new list of characters based off the old list of characters.

In [39]:
trait DisneyCharacter
case class Hero(name: String) extends DisneyCharacter
case class Villain(name: String, isTiger: Boolean) extends DisneyCharacter
case class SideKick(name: String) extends DisneyCharacter

val jungleBook = Seq(
    new Hero("Mowgli"),
    new Villain("Shere Khan", true),
    new SideKick("Baloo"),
    new SideKick("Bagheera"),
    new Villain("Kaa", false)
)

val lionKing = Seq(
    new Hero("Simba"),
    new Villain("Scar", true),
    new SideKick("Zazu"),
    new SideKick("Timone"),
    new SideKick("Pumba"),
    new Villain("Hyeena", false)
)

def teaser(chars: Seq[DisneyCharacter]): String = {
    val heros = chars.collect { case h: Hero => h.name }.mkString(" and ")
    val sidekicks = chars.collect { case s: SideKick => s.name }.mkString(" and ")
    val villains = chars.collect { case v: Villain => v.name }.mkString(" and ")
    s"Watch the courageous $heros, along with $sidekicks, defeat $villains!!!"
}

def sequel(chars: Seq[DisneyCharacter]): Seq[DisneyCharacter] = chars.map {
    _ match {
        case Hero(name) => SideKick(name)              // You already got your own movie!
        case SideKick("Zazu") => SideKick("Zazu")      // Once a king's advisor, always a king's advisor 
        case SideKick(name) => Hero(name)              // Our new heroes!
        case Villain(name, true) => Villain(name, true)// Cats always make great villains...
        case Villain(name, _) => SideKick(name)        // Bad guys who get a chance for redemption!
    }
}

println(teaser(jungleBook))
println(teaser(sequel(jungleBook)))
println(teaser(lionKing))
println(teaser(sequel(lionKing)))

Watch the courageous Mowgli, along with Baloo and Bagheera, defeat Shere Khan and Kaa!!!
Watch the courageous Baloo and Bagheera, along with Mowgli and Kaa, defeat Shere Khan!!!
Watch the courageous Simba, along with Zazu and Timone and Pumba, defeat Scar and Hyeena!!!
Watch the courageous Timone and Pumba, along with Simba and Zazu and Hyeena, defeat Scar!!!


defined trait DisneyCharacter
defined class Hero
defined class Villain
defined class SideKick
jungleBook: Seq[Product with Serializable with DisneyCharacter] = List(
  Hero(Mowgli),
  Villain(Shere Khan,true),
  SideKick(Baloo),
  SideKick(Bagheera),
  Villain(Kaa,false)
)
lionKing: Seq[Product with Serializable with DisneyCharacter] = List(
  Hero(Simba),
  Villain(Scar,true),
  SideKick(Zazu),
  SideKick(Timone),
  SideKick(Pumba),
  Villain(Hyeena,false)
)
defined function teaser
defined function sequel

If you look at the `sequel` function, you should note that addition to matching on the type of each character, we are also:
- Directly reference internal values of each character
- Sometimes, are matching directly on the internal values of each character

These are possible due to the compiler implementing an `unapply` method. Note that unapplying the case is just syntactic sugar; e.g. the following two cases examples are equivalent:
```scala
case h: Hero => SideKick(h.name)
case Hero(name) => SideKick(name)
```

In addition, there are more syntaxes and styles of matching. The following two cases are also equivalent, but the second allows you to match on internal values while still reference the parent value:
```scala
case Villain(name, true) => Villain(name, true)
case v@Villain(name, true) => v
```

Finally, you can directly embed condition checking into match statements, as demonstrated by the third of these equivalent examples:
```scala
case SideKick("Zazu") => SideKick("Zazu")
case s@SideKick("Zazu") => s
case s: SideKick if s.name == "Zazu" => s
```

All these syntaxes are enabled by a Scala unapply method contained in a class's companion object. If you want to unapply a class but do not want to make it a case class, you can manually implement the unapply method. The following example demonstrates how one can manually implement a class's apply and unapply methods:

In [ ]:
class Boat(val name: String, val length: Int)
object Boat {
    def unapply(b: Boat): Option[(String, Int)] = Some((b.name, b.length))
    def apply(name: String, length: Int): Boat = new Boat(name, length)
}

def getSmallBoats(seq: Seq[Boat]): Seq[Boat] = seq.filter { b =>
    b match {
        case Boat(_, length) if length < 60 => true
        case Boat(_, _) => false
    }
}

val boats = Seq(Boat("Santa Maria", 62), Boat("Pinta", 56), Boat("Nina", 50))
println(getSmallBoats(boats).map(_.name).mkString(" and ") + " are small boats!")

***Great job getting through the entire tutorial!!***